In [ ]:
from langchain.tools import Tool
import json
from langchain_openai import ChatOpenAI
import os

def suggest_supplier_tool(rawmaterial_id, ratingcondition=True):
    
    if not rawmaterial_id.startswith("rawmaterial_id = "):
        return "Please provide an raw material ID in the format: 'rawmaterial_id = <your_rawmaterial_id>'. for the correct format, don't ask the user again. you prepare the correct format if you have enough information."
        
    # Load JSON data from a file
    with open('suppliers.json', 'r') as file:
        data = json.load(file)
        
    rawmaterial_id = rawmaterial_id.replace("rawmaterial_id = ", "")

    if rawmaterial_id in data:
        suppliers = data[rawmaterial_id]
        if ratingcondition:
            maxrating = 0
            # Find the maximum rating
            # YOUR CODE HERE
            for i in range(len(suppliers)):
                maxrating = max(maxrating,suppliers[i]["rating"])
            # Filter suppliers with the maximum rating
            # YOUR CODE HERE
            maxratedsuppliers = [i for i in suppliers if i["rating"]==maxrating]
            # Select the supplier with the lowest price among the best rated
            lowpriced = maxratedsuppliers[0]
            for i in maxratedsuppliers:
                if(i["price"]<lowpriced["price"]):
                    lowpriced = i
            selected_supplier = lowpriced
        else:
            lowestprice = suppliers[0]
            for i in suppliers:
                if(i["price"]<lowestprice["price"]):
                    lowestprice = i
            selected_supplier = lowestprice
        
        return {
            "rawmaterial_id": rawmaterial_id,
            "supplier-id": selected_supplier["supplier-id"],
            "price": selected_supplier["price"],
            "rating": selected_supplier["rating"]
        }
    else:
        return None

In [ ]:
def get_inventory_info_tool(rawmaterial_id):
    if not rawmaterial_id.startswith("rawmaterial_id = "):
        return "Please provide an raw material ID in the format: 'rawmaterial_id = <your_rawmaterial_id>'. for the correct format, don't ask the user again. you prepare the correct format if you have enough information."
    
    try:
        rawmaterial_id = rawmaterial_id.replace("rawmaterial_id = ", "")
        with open('inventory.json', 'r') as file:
            inventory = json.load(file)
        return inventory.get(str(rawmaterial_id), "raw material ID not found")
    except Exception as e:
        print(f"Error reading orders.json: {e}")
        return "raw material ID not found"

In [ ]:
input_tool = Tool(
    name="get_input_from_user",
    func=input,
    description="Get the input from the user."
)

suggest_supplier_tool = Tool(
    name="suggest_supplier",
    func=suggest_supplier_tool,
    description="Get the supplier for a give raw material id"
)

inventory_info_tool = Tool(
    name="get_inventory_info",
    func=get_inventory_info_tool,
    description="Get information of Inventory for a given raw meterial ID."
)

In [ ]:

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.llms import OpenAI
from langchain.document_loaders import PyMuPDFLoader

# Load and index documents
loader = TextLoader("Bill_of_Material.txt")
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
split_docs = text_splitter.split_documents(documents)

# Create embeddings and build FAISS index
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(split_docs, embeddings)
retriever = vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.2})

from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, load_tools

tool_search = create_retriever_tool(
    retriever=retriever,
    name="Bill-of-Materials",
    description="Searches and bill of materials for each product"
)

tools = load_tools([], llm=llm)
tools.append(tool_search)
tools.append(input_tool)
tools.append(inventory_info_tool)
tools.append(suggest_supplier_tool)

from langchain import hub
prompt = hub.pull("hwchase17/react")

# only creates the logical steps for us
from langchain.agents import AgentExecutor, create_react_agent

react_agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=react_agent, tools=tools, verbose=True, handle_parsing_errors=True
)

def prompt_user_for_info(query):
    response = agent_executor.invoke({"input": query})
    if "Please provide" in response["output"]:
        new_input = input(response["output"])  # Prompt the user for additional input
        return prompt_user_for_info(new_input)
    else:
        return response

In [ ]:
query = "Can you give me the bill of materials?"
final_response = prompt_user_for_info(query)
print(final_response)

In [ ]:
query = "What is the current inventory of raw materials needed for P1"
final_response = prompt_user_for_info(query)
print(final_response)

In [ ]:
query = "Can you suggest suppliers who can supply raw materials needed to manufacture product P1?"
final_response = prompt_user_for_info(query)
print(final_response)